In [48]:
%load_ext autoreload
%autoreload 2

In [38]:
from dataset import SyntheticDataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from training_data import load_training_data
import torch
from tqdm import tqdm

dataset_path = (
    r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\datasets"
    r"\synthetic\synth_dataset_large.pkl"
    )

sampling_rate = 1.0

X, y = load_training_data(
        dataset_path, column_x='Squiggle', column_y='Motifs',
        sampling_rate=sampling_rate)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=1)

Index(['Squiggle', 'Bases', 'read_id', 'Motifs'], dtype='object')


In [ ]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.nn import CTCLoss
from sklearn.preprocessing import normalize

[autoreload of nn failed: Traceback (most recent call last):
  File "c:\Users\Parv\anaconda3\envs\pytorch_gpu\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\Parv\anaconda3\envs\pytorch_gpu\Lib\site-packages\IPython\extensions\autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "c:\Users\Parv\anaconda3\envs\pytorch_gpu\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "c:\Users\Parv\anaconda3\envs\pytorch_gpu\Lib\site-packages\IPython\extensions\autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Parv\anaconda3\envs\pytorch_gpu\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "c:\Users\Parv\anaconda3\envs\pytorch_gpu\Lib\site-packages\IPython\extensions\autoreload.py", line

In [40]:
df = pd.read_pickle(r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\data\synthetic\pickled_datasets\25_2_25.pkl")

In [41]:
X = df['squiggle']
Y = df['motif_seq']

In [42]:
ctc = CTCLoss(blank=0)

In [113]:
import torch.optim as optim
from nn import MotifCaller
model = MotifCaller(n_classes=16)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [117]:
ind = 0
for x, y in zip(X, Y):
    len_x = len(x)
    x = torch.tensor(x, dtype=torch.float32).view(1, len_x)
    x = normalize(x)
    print(x.shape)
    x = torch.tensor(x, dtype=torch.float32).view(1, 1, len_x)
    y = torch.tensor(y)
    model_output = model(x)
    #print(model_output.shape)
    #sum_labels = model_output.sum(axis=1)[0].detach().numpy()
    #print(np.exp(sum_labels))
    
    n_timesteps = model_output.shape[1]
    input_lengths = torch.tensor(n_timesteps)
    label_lengths = torch.tensor(len(y))

    loss = ctc(model_output[0], y, input_lengths, label_lengths)
    print(loss.item())
    #print(loss)
    loss.backward()
    optimizer.step()
    ind += 1
    
    #print([np.exp(i) for i in ]])
    if ind == 10:
        break

(1, 8034)
13.123896598815918
(1, 3415)
1.0524003505706787
(1, 9337)
3.56681227684021
(1, 7345)
5.185147285461426
(1, 7380)
6.144494533538818
(1, 11370)
7.255798816680908
(1, 3125)
3.9470643997192383
(1, 7443)
6.6799211502075195
(1, 5454)
5.731595993041992
(1, 3386)
4.050175666809082


In [101]:
print(target_lengths)

tensor(17)


In [106]:
print(input_lengths)

tensor(50)


In [108]:
print(loss)

tensor(6.3805, grad_fn=<MeanBackward0>)


### Empirical dataset

In [18]:
dataset_path = r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\data\empirical\pickled_datasets\forward_oriented.pkl"
import pandas as pd

In [19]:
df = pd.read_pickle(dataset_path)

In [28]:
dataset_ = df.loc[
    df['Library_Motifs'].apply(lambda j: sum(len(i) for i in j) > 5)
]

In [22]:
from transcript_sorting import create_reduced_spacer_transcript

In [29]:
dataset_['motif_seq'] = dataset_['motif_seq'].apply(lambda j: create_reduced_spacer_transcript(j))

In [31]:
dataset_.to_pickle(r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\data\empirical\pickled_datasets\reduced_spacers.pkl")

In [14]:
dataset_['motif_seq'] = dataset_['motif_seq'].apply(lambda j: [i for i in j if i < 9])

In [15]:
dataset_['motif_seq']

0              [3, 5, 1, 1, 4, 5]
1        [2, 2, 1, 4, 8, 3, 3, 5]
2              [5, 6, 2, 3, 3, 7]
3              [4, 8, 2, 5, 1, 5]
4           [2, 7, 7, 5, 6, 5, 6]
                   ...           
15074          [4, 4, 1, 3, 5, 6]
15075          [4, 1, 8, 6, 6, 7]
15076          [3, 2, 4, 5, 3, 2]
15077          [4, 2, 4, 8, 1, 6]
15078          [5, 5, 8, 5, 1, 4]
Name: motif_seq, Length: 10587, dtype: object

In [16]:
dataset_.to_pickle(r'C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\data\empirical\pickled_datasets\no_spacers.pkl')

In [17]:
dataset_

,read_id,ONT_Barcode_x,HW_Address_x,orientation,start_end,library_motif,ONT_Barcode_y,HW_Address_y,Payload,Library_Motifs,squiggle,Motifs,motif_seq,Payload_Sequence
0,0804c886-cd0a-4ece-87ee-adb529974699,barcode01,barcode_external01_internal01,+|+|+|+|+|+|+,373-422|150-199|423-472|472-521|246-295|572-62...,ltm8_2x1|ltm8_3x3|ltm8_3x5|ltm8_4x1|ltm8_5x1|l...,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[3, 5], [1], [1], [4], [5], [], [], []]","[0.7250000000000001, 0.7350000000000001, 0.752...","[1, 3, 5, 1, 1, 4, 5]","[3, 5, 1, 1, 4, 5]","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
1,1cc7b7bb-cd90-485f-b744-cf846d566675,barcode01,barcode_external01_internal01,+|+|+|+|+|+|+|+|+,116-165|215-264|265-314|315-364|365-414|415-46...,ltm8_1x1|ltm8_3x2|ltm8_4x2|ltm8_5x1|ltm8_6x4|l...,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[2], [2], [1], [4], [8], [3], [3], [5]]","[0.7113163972286374, 0.6628175519630484, 0.662...","[1, 2, 2, 1, 4, 8, 3, 3, 5]","[2, 2, 1, 4, 8, 3, 3, 5]","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
2,21b40579-d655-4ed0-983c-549cfe10d8af,barcode01,barcode_external01_internal01,+|+|+|+|+|+|+,134-183|279-328|329-378|379-428|429-478|479-52...,ltm8_2x1|ltm8_5x5|ltm8_6x6|ltm8_7x2|ltm8_8x3|l...,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[], [], [5], [6], [2], [3], [3], [7]]","[0.5531681113417165, 0.5592723721157368, 0.558...","[1, 5, 6, 2, 3, 3, 7]","[5, 6, 2, 3, 3, 7]","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
3,226916ad-3421-4311-9ea3-587956093e85,barcode01,barcode_external01_internal01,+|+|+|+|+|+|+|+,166-215|216-265|633-682|417-466|683-732|467-51...,ltm8_2x1|ltm8_3x4|ltm8_6x8|ltm8_7x2|ltm8_7x2|l...,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[4], [], [], [8], [2], [5], [1, 5], []]","[0.4908925318761384, 0.3770491803278688, 0.386...","[1, 4, 8, 2, 2, 5, 1, 5]","[4, 8, 2, 5, 1, 5]","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
4,446f24b0-6392-4005-b9c6-5ef7c8cece73,barcode01,barcode_external01_internal01,+|+|+|+|+|+|+,194-243|244-293|339-388|389-438|439-488|489-53...,ltm8_3x2|ltm8_4x7|ltm8_6x7|ltm8_7x5|ltm8_8x6|l...,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[2], [7], [], [7], [5], [6], [5], [6]]","[0.5530460261262361, 0.5570748382370895, 0.555...","[2, 7, 7, 5, 6, 5, 6]","[2, 7, 7, 5, 6, 5, 6]","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15074,dccf2f92-79b1-4422-86ea-42b02e683eb2,barcode73,barcode_external08_internal08,+|+|+|+|+|+|+,125-174|175-224|225-274|275-324|325-374|422-47...,ltm8_2x8|ltm8_3x4|ltm8_4x4|ltm8_5x1|ltm8_6x3|l...,73,barcode_external08_internal08,"[[3, 4, 5, 7], [2, 3, 4, 5], [1, 2, 4, 8], [3,...","[[4], [4], [1], [3], [], [5], [], [6]]","[0.5586619460383347, 0.5573190086680503, 0.556...","[8, 4, 4, 1, 3, 5, 6]","[4, 4, 1, 3, 5, 6]","[11, 3, 11, 11, 4, 11, 11, 5, 11, 11, 7, 11, 1..."
15075,e1d6beff-c03a-4bab-bfa4-a4175da41be1,barcode73,barcode_external08_internal08,+|+|+|+|+|+|+,119-168|269-318|319-368|369-418|419-468|514-56...,ltm8_1x8|ltm8_4x4|ltm8_5x1|ltm8_6x8|ltm8_7x6|l...,73,barcode_external08_internal08,"[[3, 4, 5, 7], [2, 3, 4, 5], [1, 2, 4, 8], [3,...","[[], [4], [1], [8], [6], [], [6], [7]]","[0.5864553314121037, 0.49279538904899134, 0.52...","[8, 4, 1, 8, 6, 6, 7]","[4, 1, 8, 6, 6, 7]","[11, 3, 11, 11, 4, 11, 11, 5, 11, 11, 7, 11, 1..."
15076,f566dadf-dd54-4efe-8d04-94f6dededac8,barcode73,barcode_external08_internal08,-|-|-|-|-|-|-,610-659|512-561|462-511|412-461|362-411|312-36...,ltm8_1x8|ltm8_3x3|ltm8_4x2|ltm8_5x4|ltm8_6x5|l...,73,barcode_external08_internal08,"[[3, 4, 5, 7], [2, 3, 4, 5], [1, 2, 4, 8], [3,...","[[3], [2], [4], [5], [3], [], [], [2]]","[0.7506172839506173, 0.5851851851851851, 0.612...","[8, 3, 2, 4, 5, 3, 2]","[3, 2, 4, 5, 3, 2]","[11, 3, 11, 11, 4, 11, 11, 5, 11, 11, 7, 11,

In [4]:
df.rename(columns={"Spacer_Sequence": "motif_seq"}, inplace=True)


In [ ]:
df.to_pickle(r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\data\empirical\pickled_datasets\28_2_25.pkl")

In [8]:
df

,ONT_Barcode,HW_Address,Payload,Library_Motifs,read_id,squiggle,Motifs,motif_seq,Payload_Sequence
0,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[3, 5], [1], [1], [4], [5], [], [], []]",0804c886-cd0a-4ece-87ee-adb529974699,"[0.7250000000000001, 0.7350000000000001, 0.752...","[1, 3, 5, 1, 1, 4, 5]","[11, 3, 11, 11, 5, 11, 12, 1, 12, 13, 1, 13, 1...","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
1,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[5], [1], [], [7], [7], [8], [], []]",0f041c54-7071-49d3-8ae2-7a1bf25525ab,"[0.525, 0.4845238095238096, 0.5035714285714287...","[5, 1, 1, 7, 7, 7, 8]","[11, 5, 11, 12, 1, 12, 14, 7, 14, 15, 7, 15, 1...","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
2,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[], [2], [], [4], [5], [6], [5], []]",1361a5db-d135-4e98-bb49-7a53c8d72991,"[0.6675799086757991, 0.5095890410958903, 0.515...","[1, 1, 2, 7, 4, 5, 6, 5]","[12, 2, 12, 14, 4, 14, 15, 5, 15, 16, 6, 16, 1...","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
3,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[4], [], [4], [6], [], [6], [], []]",1b4284d2-bee2-4a15-abab-aad861447308,"[0.6846153846153846, 0.5461538461538461, 0.549...","[1, 1, 1, 4, 4, 6, 6]","[11, 4, 11, 13, 4, 13, 14, 6, 14, 16, 6, 16]","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
4,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[2], [2], [1], [4], [8], [3], [3], [5]]",1cc7b7bb-cd90-485f-b744-cf846d566675,"[0.7113163972286374, 0.6628175519630484, 0.662...","[1, 2, 2, 1, 4, 8, 3, 3, 5]","[11, 2, 11, 12, 2, 12, 13, 1, 13, 14, 4, 14, 1...","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
...,...,...,...,...,...,...,...,...,...
23934,73,barcode_external08_internal08,"[[3, 4, 5, 7], [2, 3, 4, 5], [1, 2, 4, 8], [3,...","[[], [4], [1], [8], [6], [], [6], [7]]",e1d6beff-c03a-4bab-bfa4-a4175da41be1,"[0.5864553314121037, 0.49279538904899134, 0.52...","[8, 4, 1, 8, 6, 6, 7]","[12, 4, 12, 13, 1, 13, 14, 8, 14, 15, 6, 15, 1...","[11, 3, 11, 11, 4, 11, 11, 5, 11, 11, 7, 11, 1..."
23935,73,barcode_external08_internal08,"[[3, 4, 5, 7], [2, 3, 4, 5], [1, 2, 4, 8], [3,...","[[], [4], [2], [5], [1], [], [5], []]",ea27bf18-d3df-4b52-8f9e-98d23ce43b15,"[0.6136054421768707, 0.6054421768707482, 0.621...","[8, 8, 4, 2, 5, 1, 5]","[12, 4, 12, 13, 2, 13, 14, 5, 14, 15, 1, 15, 1...","[11, 3, 11, 11, 4, 11, 11, 5, 11, 11, 7, 11, 1..."
23936,73,barcode_external08_internal08,"[[3, 4, 5, 7], [2, 3, 4, 5], [1, 2, 4, 8], [3,...","[[3], [2], [4], [5], [3], [], [], [2]]",f566dadf-dd54-4efe-8d04-94f6dededac8,"[0.7506172839506173, 0.5851851851851851, 0.612...","[8, 3, 2, 4, 5, 3, 2]","[11, 3, 11, 12, 2, 12, 13, 4, 13, 14, 5, 14, 1...","[11, 3, 11, 11, 4, 11, 11, 5, 11, 11, 7, 11, 1..."
23937,73,barcode_external08_internal08,"[[3, 4, 5, 7], [2, 3, 4, 5], [1, 2, 4, 8], [3,...","[[4], [2], [4], [8], [], [1], [6], []]",f6e6ff66-98f2-4cca-9840-cedb55ee7f9d,"[0.6947115384615384, 0.7680288461538461, 0.728...","[8, 4, 2, 4, 8, 1, 6]","[11, 4, 11, 12, 2, 12, 13, 4, 13, 14, 8, 14, 1...","[11, 3, 11, 11, 4, 11, 11, 5, 11, 11, 7, 11, 1..."


In [13]:
t = df['motif_seq'].to_numpy().flatten()

In [1]:
import pandas as pd

In [4]:
df = pd.read_csv(r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\data\empirical\motif_search_barcoded.csv")

In [5]:
df2 = pd.read_pickle(r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\data\empirical\pickled_datasets\filtered_6.pkl")

In [7]:
read_ids = df['read_id'].tolist()

In [9]:
read_ids_filtered = df2['read_id'].tolist()

In [11]:
print(len(set(read_ids).intersection(set(read_ids_filtered))))

15079


In [14]:
df_ = df[df['read_id'].isin(read_ids_filtered)]

In [15]:
merged_df = pd.merge(df_, df2, on='read_id')

In [17]:
merged_df = merged_df[df['orientation'].str.startswith('+')]

C:\Users\Parv\AppData\Local\Temp\ipykernel_3416\615474548.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  merged_df = merged_df[df['orientation'].str.startswith('+')]


In [20]:

import math
from typing import List

def create_motif_label(motif_seq: List[int]) -> List[int]:
    """ 12 4 12 12 3 12 -> 12 4 2 3 4 12 13 2 4 5 3 13"""

    spacer_seq = [i for i in motif_seq if i > 8]
    starting_cycle = spacer_seq[0]
    ending_cycle = spacer_seq[-1]

    payload_seq = [i for i in motif_seq if i < 9]
    
    starting_seq_len = int(math.ceil(spacer_seq.count(starting_cycle) / 2))
    ending_seq_len = int(math.ceil(spacer_seq.count(ending_cycle) / 2))

    motif_seq_ = []
    cycle_number = starting_cycle
    ptr = 0
    while True:
        
        motif_seq_.append(cycle_number)
        if cycle_number == starting_cycle:
            motif_seq_.extend(payload_seq[: starting_seq_len])
            ptr += starting_seq_len
        elif cycle_number == ending_cycle:
            motif_seq_.extend(payload_seq[-ending_seq_len:])
        else:
            motif_seq_.extend(payload_seq[ptr: ptr + 4])
            ptr += 4
            
        motif_seq_.append(cycle_number)
        cycle_number += 1

        if cycle_number > ending_cycle:
            break


    return motif_seq_

In [21]:
motif_seq = merged_df['motif_seq'].tolist()

In [19]:
len(merged_df)

10587

In [26]:
ms_reduced_spacers = [create_motif_label(i) for i in motif_seq]

In [27]:
ms_reduced_spacers

[[11, 3, 5, 11, 12, 1, 1, 4, 5, 12, 13, 13, 14, 14, 15, 5, 15],
 [11,
  2,
  11,
  12,
  2,
  1,
  4,
  8,
  12,
  13,
  3,
  3,
  5,
  13,
  14,
  14,
  15,
  15,
  16,
  16,
  17,
  17,
  18,
  5,
  18],
 [13, 5, 13, 14, 6, 2, 3, 3, 14, 15, 7, 15, 16, 16, 17, 17, 18, 7, 18],
 [11,
  4,
  11,
  12,
  8,
  2,
  5,
  1,
  12,
  13,
  5,
  13,
  14,
  14,
  15,
  15,
  16,
  16,
  17,
  1,
  5,
  17],
 [11,
  2,
  11,
  12,
  7,
  7,
  5,
  6,
  12,
  13,
  5,
  6,
  13,
  14,
  14,
  15,
  15,
  16,
  16,
  17,
  17,
  18,
  6,
  18],
 [11,
  5,
  11,
  12,
  1,
  6,
  7,
  1,
  12,
  13,
  6,
  13,
  14,
  14,
  15,
  15,
  16,
  16,
  17,
  17,
  18,
  6,
  18],
 [11,
  5,
  11,
  12,
  1,
  6,
  4,
  7,
  12,
  13,
  6,
  6,
  13,
  14,
  14,
  15,
  15,
  16,
  16,
  17,
  17,
  18,
  6,
  18],
 [13, 1, 13, 14, 8, 7, 8, 3, 14, 15, 6, 15, 16, 3, 6, 16],
 [12,
  7,
  12,
  13,
  5,
  4,
  2,
  8,
  13,
  14,
  5,
  7,
  14,
  15,
  15,
  16,
  16,
  17,
  17,
  18,
  7,
  18],
 [11, 3

In [28]:
merged_df.columns

Index(['read_id', 'ONT_Barcode_x', 'HW_Address_x', 'orientation', 'start_end',
       'library_motif', 'ONT_Barcode_y', 'HW_Address_y', 'Payload',
       'Library_Motifs', 'squiggle', 'Motifs', 'motif_seq',
       'Payload_Sequence'],
      dtype='object')

In [29]:
merged_df.to_pickle(r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\data\empirical\pickled_datasets\forward_oriented.pkl")

In [30]:
merged_df

,read_id,ONT_Barcode_x,HW_Address_x,orientation,start_end,library_motif,ONT_Barcode_y,HW_Address_y,Payload,Library_Motifs,squiggle,Motifs,motif_seq,Payload_Sequence
0,0804c886-cd0a-4ece-87ee-adb529974699,barcode01,barcode_external01_internal01,+|+|+|+|+|+|+,373-422|150-199|423-472|472-521|246-295|572-62...,ltm8_2x1|ltm8_3x3|ltm8_3x5|ltm8_4x1|ltm8_5x1|l...,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[3, 5], [1], [1], [4], [5], [], [], []]","[0.7250000000000001, 0.7350000000000001, 0.752...","[1, 3, 5, 1, 1, 4, 5]","[11, 3, 11, 11, 5, 11, 12, 1, 12, 13, 1, 13, 1...","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
1,1cc7b7bb-cd90-485f-b744-cf846d566675,barcode01,barcode_external01_internal01,+|+|+|+|+|+|+|+|+,116-165|215-264|265-314|315-364|365-414|415-46...,ltm8_1x1|ltm8_3x2|ltm8_4x2|ltm8_5x1|ltm8_6x4|l...,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[2], [2], [1], [4], [8], [3], [3], [5]]","[0.7113163972286374, 0.6628175519630484, 0.662...","[1, 2, 2, 1, 4, 8, 3, 3, 5]","[11, 2, 11, 12, 2, 12, 13, 1, 13, 14, 4, 14, 1...","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
2,21b40579-d655-4ed0-983c-549cfe10d8af,barcode01,barcode_external01_internal01,+|+|+|+|+|+|+,134-183|279-328|329-378|379-428|429-478|479-52...,ltm8_2x1|ltm8_5x5|ltm8_6x6|ltm8_7x2|ltm8_8x3|l...,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[], [], [5], [6], [2], [3], [3], [7]]","[0.5531681113417165, 0.5592723721157368, 0.558...","[1, 5, 6, 2, 3, 3, 7]","[13, 5, 13, 14, 6, 14, 15, 2, 15, 16, 3, 16, 1...","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
3,226916ad-3421-4311-9ea3-587956093e85,barcode01,barcode_external01_internal01,+|+|+|+|+|+|+|+,166-215|216-265|633-682|417-466|683-732|467-51...,ltm8_2x1|ltm8_3x4|ltm8_6x8|ltm8_7x2|ltm8_7x2|l...,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[4], [], [], [8], [2], [5], [1, 5], []]","[0.4908925318761384, 0.3770491803278688, 0.386...","[1, 4, 8, 2, 2, 5, 1, 5]","[11, 4, 11, 14, 8, 14, 15, 2, 15, 16, 5, 16, 1...","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
4,446f24b0-6392-4005-b9c6-5ef7c8cece73,barcode01,barcode_external01_internal01,+|+|+|+|+|+|+,194-243|244-293|339-388|389-438|439-488|489-53...,ltm8_3x2|ltm8_4x7|ltm8_6x7|ltm8_7x5|ltm8_8x6|l...,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[2], [7], [], [7], [5], [6], [5], [6]]","[0.5530460261262361, 0.5570748382370895, 0.555...","[2, 7, 7, 5, 6, 5, 6]","[11, 2, 11, 12, 7, 12, 14, 7, 14, 15, 5, 15, 1...","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15074,dccf2f92-79b1-4422-86ea-42b02e683eb2,barcode73,barcode_external08_internal08,+|+|+|+|+|+|+,125-174|175-224|225-274|275-324|325-374|422-47...,ltm8_2x8|ltm8_3x4|ltm8_4x4|ltm8_5x1|ltm8_6x3|l...,73,barcode_external08_internal08,"[[3, 4, 5, 7], [2, 3, 4, 5], [1, 2, 4, 8], [3,...","[[4], [4], [1], [3], [], [5], [], [6]]","[0.5586619460383347, 0.5573190086680503, 0.556...","[8, 4, 4, 1, 3, 5, 6]","[11, 4, 11, 12, 4, 12, 13, 1, 13, 14, 3, 14, 1...","[11, 3, 11, 11, 4, 11, 11, 5, 11, 11, 7, 11, 1..."
15075,e1d6beff-c03a-4bab-bfa4-a4175da41be1,barcode73,barcode_external08_internal08,+|+|+|+|+|+|+,119-168|269-318|319-368|369-418|419-468|514-56...,ltm8_1x8|ltm8_4x4|ltm8_5x1|ltm8_6x8|ltm8_7x6|l...,73,barcode_external08_internal08,"[[3, 4, 5, 7], [2, 3, 4, 5], [1, 2, 4, 8], [3,...","[[], [4], [1], [8], [6], [], [6], [7]]","[0.5864553314121037, 0.49279538904899134, 0.52...","[8, 4, 1, 8, 6, 6, 7]","[12, 4, 12, 13, 1, 13, 14, 8, 14, 15, 6, 15, 1...","[11, 3, 11, 11, 4, 11, 11, 5, 11, 11, 7, 11, 1..."
15076,f566dadf-dd54-4efe-8d04-94f6dededac8,barcode73,barcode_external08_internal08,-|-|-|-|-|-|-,610-659|512-561|462-511|412-461|362-411|312-36...,ltm8_1x8|ltm8_3x3|ltm8_4x2|ltm8_5x4|ltm8_6x5|l...,73,barcode_external08_internal08,"[[3, 4, 5, 7], [2, 3, 4, 5